In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import functools
import sys
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data.dataset import Dataset
import torch.nn.functional as F
import torchtext
import tqdm
import random
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.lm import MLE
from nltk.util import ngrams
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
import re
import os
import re
import string
import json
import numpy as np
import pandas as pd
from sklearn import metrics
from bs4 import BeautifulSoup
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, AutoTokenizer, BertModel, BertConfig, AutoModel, AdamW
import warnings

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
!pip install transformers

In [3]:
class BERTtestDataset(Dataset):
    def posaddedtext(self,text):
        
        try:
            doc = nlp(text)
            for token in doc:
                text = text + " " + token.tag_
            return text
        except:
            return text
    
    def neraddedtext(self,text):
        try:
            doc = nlp(text)
            for ent in doc.ents:
                text = text + " " + ent.label_
            return text
        except:
            return text
    
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.max_len = max_len
        self.text = df.Tweet
        self.tokenizer = tokenizer
       
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.posaddedtext(self.text[index])
#         text = self.text[index]
        # text = self.neraddedtext(self.text[index])
        inputs = self.tokenizer.encode_plus(
            text,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
    
        }

In [ ]:
# To find out the current mood of the person who is making the statement out of the following label
f=['anger', 'disgust', 'fear', 'joy', 'love','sadness']

In [4]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = "cpu"

In [5]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 80
# TRAIN_BATCH_SIZE = 64
VALID_BATCH_SIZE = 1
EPOCHS = 5
LEARNING_RATE = 2e-5
tokenizer = AutoTokenizer.from_pretrained('roberta-large')

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [6]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.roberta1 = AutoModel.from_pretrained('roberta-large')
#         self.l2 = torch.nn.Dropout(0.3)
        self.fc = torch.nn.Linear(1024,6)
    
    def forward(self, ids, mask, token_type_ids):
        _, features1 = self.roberta1(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
        # _, features2 = self.roberta2(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict = False)
        # last_hidden_states = torch.cat([features1,features2],dim=1)
        output = self.fc(features1)
        return output

model = BERTClass()
model.to(device);

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
PATH = "/kaggle/input/okkkkk/last_model (3).h5"
loaded_model = BERTClass()
loaded_model.load_state_dict(torch.load(PATH,map_location=torch.device('cpu')))


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [14]:
 p = loaded_model.to(device)

In [30]:
x = "I am pleased to hear about you"
arr = []
arr.append(x)
pf = pd.DataFrame()
pf['Tweet'] = arr

valid_dataset = BERTtestDataset(pf, tokenizer, MAX_LEN)

valid_loader = DataLoader(valid_dataset, batch_size=1, 
                           num_workers=2, shuffle=False, pin_memory=True)

In [31]:
def validation():
    loaded_model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        print(valid_loader)
        for _, data in enumerate(valid_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            outputs = loaded_model(ids, mask, token_type_ids)
            
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs


In [32]:
outputs = validation()
outputs

[[0.996228814125061,
  0.9213868379592896,
  0.007203541696071625,
  0.00035084792762063444,
  0.0017582604195922613,
  0.02842453494668007]]

In [33]:
x1 = outputs[0].index(max(outputs[0]))

In [34]:
# To display the current mood the person who is making the above statement
f[x1]

'anger'